In [1]:
import os
os.chdir('../')
%pwd

'/home/asini/Workspace/Enseignement/M2/MLOps/e2e-mlops'

In [3]:
import pandas as pd

In [5]:
data = pd.read_csv('artifacts/data_ingestion/winequality-red.csv')
data.columns.to_list()

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [9]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir:Path
    STATUS_FILE : str
    unzip_data_dir :Path
    all_schema : dict

In [19]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager :
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath =  SCHEMA_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        self.schema= read_yaml(schema_filepath)

        create_directories([self.config.artifacts_roots])
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config =self.config.data_validation
        schema =self.schema.COLUMNS
        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema,
        )
        return data_validation_config


    

In [21]:
import os

from mlProject import logger

In [22]:
class DataValidation:
    def __init__(self,config: DataValidationConfig ):
        self.config = config

    def validation_all_columns(self)->bool:
        try:
            validation_status =None
            data= pd.read_csv(self.config.unzip_data_dir)
            all_cols =list(data.columns)
            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status =False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f'Validation status : {validation_status}')
                else:
                    validation_status=True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f'Validation status : {validation_status}')
            return validation_status
        except Exception as e:
            raise e

In [23]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation=DataValidation(config=data_validation_config)
    data_validation.validation_all_columns()

except Exception as e:
    raise e

[2023-10-24 16:09:05,347:INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-24 16:09:05,348:INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-24 16:09:05,350:INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-24 16:09:05,350:INFO: common: created directory at: artifacts]
[2023-10-24 16:09:05,351:INFO: common: created directory at: artifacts/data_validation]
